In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from src import caa_survey_utils, config

from src.old_mappers.ModeConditionMapperV4 import ModeConditionMapper as ModeConditionMapperV4
from src.old_mappers.ModeConditionMapperV4_Corrected import ModeConditionMapper as ModeConditionMapperV4_Corrected
from src.old_mappers.ModeConditionMapperV5 import ModeConditionMapper as ModeConditionMapperV5
from src.old_mappers.ModeConditionMapperV6 import ModeConditionMapper as ModeConditionMapperV6
from src.old_mappers.ModeConditionMapperV6_Old_LASAM_Mode_LU import ModeConditionMapper as ModeConditionMapperV6_old_lasam_mode_lu


%load_ext autoreload
%autoreload 2

c:\Users\DYCKERH\OneDrive - Jacobs\Documents\Expanding Heathrow\wip lasam mode conditions\heathrow-lasam-mode-conditions\src\old_mappers\ModeConditionMapperV4.py:6: SyntaxWarning: invalid escape sequence '\.'
  sys.path.append('..\..')


# 1. Import and Preprocess Data

## 1.1 CAA Survey Data

In [2]:
# CAA for Heathrow Airport
caa_2023_24_path = r"\\uk-lon-FAS02\Projects\UNIF\Projects\60H700SA - Heathrow SAS 2024\04 Technical\03 LASAM Development\11_Mode choice models\21 Origin assignment\01 Sent to TP\Full dataset for LASAM Zone Assignment.xlsx"
caa_original = pd.read_excel(caa_2023_24_path, engine='openpyxl')
caa_original = caa_original.drop(columns=['Date', 'Mode Group', 'Rename', 'Tylers proposed change', 'Exclusion', 'Report Method Timestamp', 'Weighting'])

### Remove Dummy Records and uplift remaining population

In [3]:
caa_lhr = caa_original[(caa_original['AIRPORT_Prefix']=='LHR')].copy()
caa_lhr.reset_index(drop=True, inplace=True)
caa_lhr = caa_survey_utils.process_dummy_records(caa_lhr)

caa_lhr.head()

0 dummy records removed and reminaing population uplifted by 1.0


,TAG,AIRPORT_Prefix,APT_TERMINAL,GATE,Year,SYSTEM_QUARTER,SYSTEM_MONTH,IDATE,SYSTEM_START_TIME,SEX,...,SYSTEM_PURPOSE1,PURPOSE,MBUSINESS,PROMOTE,ETHNIC,DISABLE,POP,DUMMY_FLAG,RUN_DATE,NATION1
0,Heathrow-11/01/2024-TZR123-330,LHR,4,NaN,2024,1,Jan,2024-01-11,13:49:35,Male,...,Leisure,Leisure Other,NaN,08,White - Irish,No,1561.885714,Missing,27/03/2025 15:07:51,Ireland
1,Heathrow-14/01/2024-IER123-328,LHR,3,NaN,2024,1,Jan,2024-01-14,06:24:19,Female,...,Business,Business,NaN,10,Other,No,809.008874,Missing,27/03/2025 15:07:51,New Zealand
2,Heathrow-05/01/2024-EFL123-326,LHR,5,NaN,2024,1,Jan,2024-01-05,10:10:41,Female,...,Leisure,Leisure Other,NaN,07,White - Any other White background,No,1058.387919,Missing,27/03/2025 15:07:51,United States of America (USA)
3,Heathrow-05/01/2024-EAU123-326,LHR,3,NaN,2024,1,Jan,2024-01-05,20:34:22,Male,...,Leisure,Visiting Friends and Relatives,NaN,06,Asian - Chinese,No,849.125000,Missing,27/03/2025 15:07:51,China
4,Heathrow-10/01/2024-IER123-314,LHR,2,NaN,2024,1,Jan,2024-01-10,17:04:42,Male,...,Leisure,Visiting Friends and Relatives,NaN,08,White - Any other White background,No,688.411796,Missing,27/03/2025 15:07:51,Sweden


### Remove records for interline passengers

In [4]:
caa_lhr = caa_survey_utils.remove_interline_pax(caa_lhr)

removed 0 rows with interline passengers


### Assign LASAM Segment

In [5]:
caa_lhr = pd.merge(caa_lhr, config.segment_lu, on=['SYSTEM_COUNTRY', 'SYSTEM_RouteTo', 'SYSTEM_PURPOSE1', 'SYSTEM_Market'], how='left')

# 2. Assign LASAM Modes

In [ ]:
# update mode fields from TfL Rail to Elizabeth Line
columns_to_update = ['MODEA', 'MODEB', 'MODEC', 'SYSTEM_FINALMODE']
caa_lhr[columns_to_update] = caa_lhr[columns_to_update].replace('TfL Rail (formerly Heathrow Connect)', 'Elizabeth Line')

# Assign LASAM mode based on CAA final mode to compare mode shares
caa_lhr = pd.merge(caa_lhr, config.caa_final_mode_lasam_mode_lu, on='SYSTEM_FINALMODE', how='left')

In [10]:
caa_lhr.columns

Index(['TAG', 'AIRPORT_Prefix', 'APT_TERMINAL', 'GATE', 'Year',
       'SYSTEM_QUARTER', 'SYSTEM_MONTH', 'IDATE', 'SYSTEM_START_TIME', 'SEX',
       'COUNTRY', 'SYSTEM_COUNTRY', 'AGE', 'AIRFROM', 'CUSTOMS', 'SYSTEM_TI',
       'SYSTEM_RouteFrom', 'SYSTEM_Region', 'SYSTEM_County', 'SYSTEM_District',
       'PCODE2', 'MODEA', 'MODEB', 'MODEC', 'SYSTEM_FINALMODE', 'ELEC_CAR',
       'JourneyTime', 'myGROUP', 'TOAIRPOR', 'SYSTEM_RouteTo', 'Fltno',
       'AIRLINE', 'FLIGHT', 'FINALAIR', 'SYSTEM_TRUEOD_AIRPORT',
       'SYSTEM_Market', 'TRIP', 'TICKET', 'SYSTEM_PURPOSE1', 'PURPOSE',
       'MBUSINESS', 'PROMOTE', 'ETHNIC', 'DISABLE', 'POP', 'DUMMY_FLAG',
       'RUN_DATE', 'NATION1', 'Segment', 'Segment_4', 'Segment_4_ID'],
      dtype='object')

In [ ]:
def apply_last_mode(row: pd.Series) -> pd.Series:
    

In [ ]:
caa_lhr['Last'] = caa_lhr.apply(lambda row: row['MODEA'] if row['MODEC'] in ['No Mode', np.NaN] and row['MODEB'] in ['No Mode', np.NaN]
                                else (row['MODEB'] if row['MODEC'] in ['No Mode', np.NaN] else row['MODEC']), axis=1)

caa_lhr['2ndLast'] = caa_lhr.apply(lambda row: 'No Mode' if row['MODEC'] in ['No Mode', np.NaN] and row['MODEB'] in ['No Mode', np.NaN]
                                   else (row['MODEA'] if row['MODEC'] in ['No Mode', np.NaN] else row['MODEB']), axis=1)

caa_lhr['3rdLast'] = caa_lhr.apply(lambda row: 'No Mode' if row['MODEC'] in ['No Mode', np.NaN] else row['MODEA'], axis=1)

caa_lhr['Origin'] = caa_lhr.apply(lambda row: 'AIRPORT' if row['SYSTEM_District'] in ['Heathrow Airport (SE)']
                                 else ('LDN' if row['SYSTEM_County']=='Greater London' else 'NonLDN'), axis=1)

caa_lhr['Contains_Elizabeth_Line'] = np.where(
    (caa_lhr['Last'] == 'Elizabeth Line') |
    (caa_lhr['2ndLast'] == 'Elizabeth Line') |
    (caa_lhr['3rdLast'] == 'Elizabeth Line'),
    True,
    False
)

caa_lhr['Contains_Heathrow_Express'] = np.where(
    (caa_lhr['Last'] == 'Heathrow Express') |
    (caa_lhr['2ndLast'] == 'Heathrow Express') |
    (caa_lhr['3rdLast'] == 'Heathrow Express'),
    True,
    False
)

caa_lhr['Contains_Tube'] = np.where(
    (caa_lhr['Last'] == 'Tube/Metro/Subway') |
    (caa_lhr['2ndLast'] == 'Tube/Metro/Subway') |
    (caa_lhr['3rdLast'] == 'Tube/Metro/Subway'),
    True,
    False
)

caa_lhr['Contains_Rental'] = np.where(
    (caa_lhr['Last'].isin(['Rental car - short term car park', 'Rental car - hire car courtesy bus'])) |
    (caa_lhr['2ndLast'].isin(['Rental car - short term car park', 'Rental car - hire car courtesy bus'])) |
    (caa_lhr['3rdLast'].isin(['Rental car - short term car park', 'Rental car - hire car courtesy bus'])),
    True,
    False
)